<a href="https://colab.research.google.com/github/pepebocio/TFG_Inteligencia-artificial-sobre-procesadores-edge-computing/blob/Auxiliares/RedDedos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import sklearn.metrics
import tensorflow as tf

from numpy import mean
from numpy import std
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import activations
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [ ]:
BATCH_SIZE = 64 ##
NUM_EPOCHS = 100 ##tiradas de entrenamiento
OUTPUT_DATASET_FILE = "modelo.csv"
TFL_MODEL_FILE = "modelo.tflite"
TFL_MODEL_HEADER_FILE = "modelo.h"
TF_MODEL = "modelo"

In [ ]:
# Abrir el archivo en modo lectura
datos = []
with open('DataSetNormalizado(1+2+3).txt', 'r') as archivo:
    # Leer cada línea del archivo
    for linea in archivo:
        # Dividir la línea en valores separados por espacios en blanco
        valores = linea.split()
        # Convertir los valores a enteros y agregarlos a la lista de datos
        datos.append([float(valor) for valor in valores])

FileNotFoundError: [Errno 2] No such file or directory: 'DataSetNormalizado(1+2+3).txt'

In [ ]:
##creamos una variable dataset que guarda 3 columnas de datos con nombres y con x, y, etiquetas

csv_header = ["dedo1","dedo2","dedo3","dedo4","out"]

df_dataset = pd.DataFrame(datos, columns = csv_header)


In [ ]:
##guardamos los nombres de las entradas y las salidas por separado
f_names = df_dataset.columns.values[0:4]
l_name  = df_dataset.columns.values[4:5]
##los usamos para guardar en X las entradas y en Y las salidas
Dedos = df_dataset[f_names]
Out = df_dataset[l_name]

onehot=tf.one_hot(Out,10)
# Convertir las etiquetas de salida a one hot encoding
num_classes = 10  # Número de clases posibles
y_one_hot = np.zeros((len(Out), num_classes))
y_one_hot[np.arange(len(Out)), Out.values.flatten().astype(int)] = 1

##separamos los datos para sus distintas funciones
# Split 1 (85% vs 15%)
x_train, x_validate_test, y_train, y_validate_test = train_test_split(Dedos, y_one_hot, test_size=0.2, random_state = 12)
# Split 2 (50% vs 50%)
x_test, x_validate, y_test, y_validate = train_test_split(x_validate_test, y_validate_test, test_size=0.50, random_state = 2)

In [ ]:
unos_por_indice = np.sum(y_validate, axis=0)

# Crear el histograma
plt.bar(np.arange(10), unos_por_indice)
plt.xlabel('Índice')
plt.ylabel('Número de unos')
plt.title('Histograma distribucion de los datos de validacion')
plt.show()

unos_por_indice = np.sum(y_test, axis=0)

plt.bar(np.arange(10), unos_por_indice)
plt.xlabel('Índice')
plt.ylabel('Número de unos')
plt.title('Histograma distribucion de los datos de validacion')
plt.show()

In [ ]:
onehot=tf.one_hot(Out,10)
type(x_train)

In [ ]:
##definimos las capas del modelo
model = tf.keras.Sequential()
model.add(layers.Dense(40, activation='relu', input_shape=(len(f_names),)))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(10, activation='relu'))

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
type(x_train)

In [ ]:
history = model.fit(x_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_validate, y_validate))

In [ ]:
loss_train = history.history['loss']
loss_val   = history.history['val_loss']
acc_train  = history.history['accuracy']
acc_val    = history.history['val_accuracy']
epochs     = range(1, NUM_EPOCHS + 1)

def plot_train_val_history(x, y_train, y_val, type_txt):
  plt.figure(figsize = (10,7))
  plt.plot(x, y_train, 'g', label='Training'+type_txt)
  plt.plot(x, y_val, 'b', label='Validation'+type_txt)
  plt.title('Training and Validation'+type_txt)
  plt.xlabel('Epochs')
  plt.ylabel(type_txt)
  plt.legend()
  plt.show()

plot_train_val_history(epochs, loss_train, loss_val, "Loss")
plot_train_val_history(epochs, acc_train, acc_val, "Accuracy")

Guardar el modelo

In [ ]:
model.save(TF_MODEL)

In [ ]:
y_test_pred = model.predict(x_test)

y_test_pred = (y_test_pred > 0.5).astype("int32")

out= np.argmax(y_test, axis=1)
##out= np.split(out,len(out))


out_pred = np.argmax(y_test_pred, axis=1)
##out_pred= np.split(out_pred,len(out_pred))

cm = sklearn.metrics.confusion_matrix(out, out_pred)

index_names  = ["Actual 0", "Actual 1", "Actual 2", "Actual 3", "Actual 4", "Actual 5", "Actual 6", "Actual 7", "Actual 8", "Actual 9"]
column_names = ["Predicted 0", "Predicted 1", "Predicted 2", "Predicted 3", "Predicted 4", "Predicted 5", "Predicted 6", "Predicted 7", "Predicted 8", "Predicted 9"]

df_cm = pd.DataFrame(cm, index = index_names, columns = column_names)

plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True, fmt='d', cmap="YlOrBr")
plt.figure(figsize = (10,7))

Comprobamos que tal funciona (comparar con las graficas del dataset)

In [ ]:
muestras=20

simulacion=np.zeros((muestras**4,5))

for i in range(muestras**4):
  simulacion[i,0]=(i%muestras)/muestras
  simulacion[i,1]=(i//muestras%muestras)/muestras
  simulacion[i,2]=(i//muestras**2%muestras)/muestras
  simulacion[i,3]=(i//muestras**3%muestras)/muestras
simulacion[:,4]=np.argmax(model.predict(simulacion[:,0:4]),axis=1)
simulacion= sorted(simulacion, key=lambda x: x[4])

In [ ]:
grupos = [dato[4] for dato in simulacion]
posiciones_dedo1 = [dato[0] for dato in simulacion]
posiciones_dedo2 = [dato[1] for dato in simulacion]
posiciones_dedo3 = [dato[2] for dato in simulacion]
posiciones_dedo4 = [dato[3] for dato in simulacion]

# Suponiendo que tienes los datos ordenados en una lista llamada datos_ordenados
# Extraer las posiciones de los dedos para cada posición y agruparlas por valor en la primera columna
posiciones_dedo1_por_grupo = {}
posiciones_dedo2_por_grupo = {}
posiciones_dedo3_por_grupo = {}
posiciones_dedo4_por_grupo = {}

for dato in simulacion:
    grupo = dato[4]
    pos_dedo1 = dato[0]
    pos_dedo2 = dato[1]
    pos_dedo3 = dato[2]
    pos_dedo4 = dato[3]

    if grupo not in posiciones_dedo1_por_grupo:
        posiciones_dedo1_por_grupo[grupo] = []
        posiciones_dedo2_por_grupo[grupo] = []
        posiciones_dedo3_por_grupo[grupo] = []
        posiciones_dedo4_por_grupo[grupo] = []

    posiciones_dedo1_por_grupo[grupo].append(pos_dedo1)
    posiciones_dedo2_por_grupo[grupo].append(pos_dedo2)
    posiciones_dedo3_por_grupo[grupo].append(pos_dedo3)
    posiciones_dedo4_por_grupo[grupo].append(pos_dedo4)

# Graficar diagrama de caja para cada posición de los dedos en gráficos separados
plt.figure(figsize=(12, 8))

plt.subplot(221)
plt.boxplot(posiciones_dedo1_por_grupo.values(), labels=posiciones_dedo1_por_grupo.keys())
plt.title('Dedo 1')
plt.xlabel('Posición')
plt.ylabel('Medida del ADC')

plt.subplot(222)
plt.boxplot(posiciones_dedo2_por_grupo.values(), labels=posiciones_dedo2_por_grupo.keys())
plt.title('Dedo 2')
plt.xlabel('Posición')
plt.ylabel('Medida del ADC')


plt.subplot(223)
plt.boxplot(posiciones_dedo3_por_grupo.values(), labels=posiciones_dedo3_por_grupo.keys())
plt.title('Dedo 3')
plt.xlabel('Posición')
plt.ylabel('Medida del ADC')

plt.subplot(224)
plt.boxplot(posiciones_dedo4_por_grupo.values(), labels=posiciones_dedo4_por_grupo.keys())
plt.title('Dedo 4')
plt.xlabel('Posición')
plt.ylabel('Medida del ADC')


plt.tight_layout()
plt.show()

In [ ]:
datos= sorted(datos, key=lambda x: x[4])
[fila[:4] for fila in datos]
resultados = np.argmax(model.predict([fila[:4] for fila in datos]), axis=1)

# Asignar los resultados a la quinta columna de datos
for i in range(len(datos)):
    datos[i][-1] = resultados[i]
datos


In [ ]:
grupos = [dato[4] for dato in datos]
posiciones_dedo1 = [dato[0] for dato in datos]
posiciones_dedo2 = [dato[1] for dato in datos]
posiciones_dedo3 = [dato[2] for dato in datos]
posiciones_dedo4 = [dato[3] for dato in datos]

# Suponiendo que tienes los datos ordenados en una lista llamada datos_ordenados
# Extraer las posiciones de los dedos para cada posición y agruparlas por valor en la primera columna
posiciones_dedo1_por_grupo = {}
posiciones_dedo2_por_grupo = {}
posiciones_dedo3_por_grupo = {}
posiciones_dedo4_por_grupo = {}

for dato in datos:
    grupo = dato[4]
    pos_dedo1 = dato[0]
    pos_dedo2 = dato[1]
    pos_dedo3 = dato[2]
    pos_dedo4 = dato[3]

    if grupo not in posiciones_dedo1_por_grupo:
        posiciones_dedo1_por_grupo[grupo] = []
        posiciones_dedo2_por_grupo[grupo] = []
        posiciones_dedo3_por_grupo[grupo] = []
        posiciones_dedo4_por_grupo[grupo] = []

    posiciones_dedo1_por_grupo[grupo].append(pos_dedo1)
    posiciones_dedo2_por_grupo[grupo].append(pos_dedo2)
    posiciones_dedo3_por_grupo[grupo].append(pos_dedo3)
    posiciones_dedo4_por_grupo[grupo].append(pos_dedo4)

# Graficar diagrama de caja para cada posición de los dedos en gráficos separados
plt.figure(figsize=(12, 8))

plt.subplot(221)
plt.boxplot(posiciones_dedo1_por_grupo.values(), labels=posiciones_dedo1_por_grupo.keys())
plt.title('Dedo 1')
plt.xlabel('Posición')
plt.ylabel('Medida del ADC')

plt.subplot(222)
plt.boxplot(posiciones_dedo2_por_grupo.values(), labels=posiciones_dedo2_por_grupo.keys())
plt.title('Dedo 2')
plt.xlabel('Posición')
plt.ylabel('Medida del ADC')


plt.subplot(223)
plt.boxplot(posiciones_dedo3_por_grupo.values(), labels=posiciones_dedo3_por_grupo.keys())
plt.title('Dedo 3')
plt.xlabel('Posición')
plt.ylabel('Medida del ADC')

plt.subplot(224)
plt.boxplot(posiciones_dedo4_por_grupo.values(), labels=posiciones_dedo4_por_grupo.keys())
plt.title('Dedo 4')
plt.xlabel('Posición')
plt.ylabel('Medida del ADC')


plt.tight_layout()
plt.show()

In [ ]:
def representative_data_gen():
  for i_value in tf.data.Dataset.from_tensor_slices(x_test).batch(1).take(50):
    i_value_f32 = tf.dtypes.cast(i_value, tf.float32)
    yield [i_value_f32]


converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)


converter.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8


tflite_model_quant = converter.convert()
open(TFL_MODEL_FILE, "wb").write(tflite_model_quant)

In [ ]:
!apt-get update && apt-get -qq install xxd
!xxd -i modelo.tflite > model.h
!cat model.h

Pruebas del modelo

In [ ]:
x=np.array([[0,1,1,1]])
model.predict(x)
np.argmax(model.predict(x), axis=1)

In [ ]:
resultado=model.predict(x_test)
sum=np.zeros(len(resultado))
for i in range(len(resultado)):
  sum[i]=np.sum(resultado[i])
# Obtener el índice del valor máximo en cada fila
indices_maximos = np.argmax(resultado, axis=1)

# Crear una matriz con la misma forma que 'predicciones' pero con 0 en todas partes excepto en los índices máximos
predicciones_con_maximos = np.zeros_like(resultado)
predicciones_con_maximos[np.arange(len(resultado)), indices_maximos] = 1

##indices_maximos
resultado

In [ ]:
indices_maximos

In [ ]:
np.argmax(y_test, axis=1)

In [ ]:
x_test
